Project for Differential Equations laboratories at AGH CS, 2018/2019 
<br>
L-Shape FEM solver

In [1]:
import numpy as np
import math
from fractions import Fraction

Function of heat at the Neumann border of L-Shape (later multiplied by user-specified coefficient)

In [2]:
g = lambda coords: math.sqrt(coords[0]**2 + coords[1]**2)

Mapping of vertex indices to coordinates for the base square (0,1) x (0,1)

In [3]:
vertex_coords = [(0,0), (1,0), (1,1), (0,1)]

Mapping of global vertex coordinates to vertex index

In [4]:
vertex_index_mapping = {
    (-1,1): 0, (0,1): 1, (1,1): 2, (-1,0): 3, (0,0): 4, (1,0): 5, (0,-1): 6, (1,-1): 7
}

Definition of local basis functions

In [5]:
base_fun_0 = lambda coords : (1-coords[0])*(1-coords[1])
base_fun_1 = lambda coords : coords[0]*(1-coords[1])
base_fun_2 = lambda coords : coords[0]*coords[1]
base_fun_3 = lambda coords : (1-coords[0])*coords[1]

Mapping of vertex coordinates to basis functions and their partial derivatives

In [6]:
base_fun = {
        vertex_coords[0] : base_fun_0,
        vertex_coords[1] : base_fun_1,
        vertex_coords[2] : base_fun_2,
        vertex_coords[3] : base_fun_3,
}

base_x_derivative = {
        vertex_coords[0] : -1,
        vertex_coords[1] : 1,
        vertex_coords[2] : 1,
        vertex_coords[3] : -1,
}

base_y_derivative = {
        vertex_coords[0] : -1,
        vertex_coords[1] : -1,
        vertex_coords[2] : 1,
        vertex_coords[3] : 1,
}

A class for representing 3 squares of size 1x1, to which L-Shape is partitioned. Each square can be represended as a pair of coordinates of it's lower-left vertex

In [7]:
class BasicParitionSquare(object):
    def __init__(self, cords):
        (x,y)=cords                       
        self.x = x
        self.y = y
     
    # Translate coordinates from inside the square to coordinates inside the basis square
    def to_base_square_coords(self, cords):
        (x,y)=cords
        return (x - self.x, y - self.y)
    
    # Reverse translation
    def from_base_square_coords(self, cords):
        (x,y)=cords
        return (x + self.x, y + self.y)
    
    def base_functions(self):
        return {
            self.from_base_square_coords(vertex_coords[0]): (lambda x: base_fun_0(self.to_base_square_coords(x))),
            self.from_base_square_coords(vertex_coords[1]): (lambda x: base_fun_1(self.to_base_square_coords(x))),
            self.from_base_square_coords(vertex_coords[2]): (lambda x: base_fun_2(self.to_base_square_coords(x))),
            self.from_base_square_coords(vertex_coords[3]): (lambda x: base_fun_3(self.to_base_square_coords(x)))
        }
    
    def x_derivatives(self):
        return{
            self.from_base_square_coords(coords): dx for coords,dx in base_x_derivative.items() 
        }
    
    def y_derivatives(self):
        return{
            self.from_base_square_coords(coords): dy for coords,dy in base_y_derivative.items()
        }
     
    def neumann_edges(self):
        result = []
        vertices = list(self.base_functions().keys())
        is_neumann = lambda coords: coords[0] == -1 or coords[0] == 1 or coords[1] == 1 or coords[1] == -1
        is_edge = (lambda c1, c2: (c1[0]==c2[0] and (c1[1]==c2[1]+1 or c1[1]==c2[1]-1)) 
        or (c1[1]==c2[1] and (c1[0]==c2[0]+1 or c1[0]==c2[0]-1)) )
        
        for i,coords in enumerate(vertices):
            for inner_coords in vertices[i+1:]:
                if is_neumann(coords) and is_neumann(inner_coords) and is_edge(coords, inner_coords):
                    result.append((coords, inner_coords))
        return result


In [8]:
E1 = BasicParitionSquare((-1,0))
E2 = BasicParitionSquare((0,0))
E3 = BasicParitionSquare((0,-1))
partition_squares = [E1,E2,E3]

self-written gaussian elimination algorithm

In [9]:
def gaussian_elimination(A, B):
    n = len(A)
    for i in range(n - 1):
        pivot = i
        for j in range(i + 1, n):
            if abs(A[j][i]) > abs(A[pivot][i]):
                pivot = j
        if pivot != i:
            # for some reason, simply swapping rows doesn't work with this matrix. Spent 3 hours figuring this
            for k in range(len(A)):
                A[i][k], A[pivot][k] = A[pivot][k], A[i][k]
            B[i], B[pivot] = B[pivot], B[i]
        
        for j in range(i + 1, n):
            multiply_row_by = np.float128(A[j][i])/np.float128(A[i][i])
            A[j][i] = 0.0
            for k in range(i + 1, n):
                A[j][k] -= multiply_row_by * np.float128(A[i][k])
            B[j] -= multiply_row_by * np.float128(B[i])
            
    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n):
            prev_solution_coefficient = A[i][j]
            B[i] -= prev_solution_coefficient * B[j]
        multiply_b_by = 1.0/np.float128(A[i][i])
        B[i] *= multiply_b_by
    return B


Function to get coefficients for the 8 base functions, to approximate desired *u* heat flow function

In [10]:
def calculate_u_coefficient_vec(g_coeff):
    B = np.zeros((8,8))
    L = np.zeros(8)
    for square in partition_squares:
        for coords, base_fun in square.base_functions().items():
            vert_num = vertex_index_mapping[coords]
            for edge in square.neumann_edges():
                (x1,y1), (x2,y2) = edge
                middle_coords = (Fraction(x1+x2, 2), Fraction(y1+y2, 2))
                # integral approximation, edge length is omitted cause is equal to 1
                L[vert_num] += g_coeff * float(g(middle_coords)) * float(base_fun(middle_coords))
            for inner_coords, inner_base_fun in square.base_functions().items():
                inn_vert_num = vertex_index_mapping[inner_coords]
                #integ. app.,  area = 1, derivarives at the centre are always 1/2 of derivatives at vertices
                B[vert_num, inn_vert_num] += \
                (square.x_derivatives()[coords] * square.x_derivatives()[inner_coords]) * Fraction(1,2)
                B[vert_num, inn_vert_num] += \
                (square.y_derivatives()[coords] * square.y_derivatives()[inner_coords]) * Fraction(1,2)
    # enforcing drichlet conditions
    for i in (3,4,6): 
        B[i,:]=0
        B[i,i]=1
        L[i]=0
    return gaussian_elimination(B,L)

Getting matrix of _u_ values in n $*$ n evenly spaced points, for a given *g* function coefficient 

In [11]:
def get_sample_values(n, g_coeff):
    U = calculate_u_coefficient_vec(g_coeff)
    x_samples = np.linspace(-1,1,n)
    y_samples = np.linspace(-1,1,n)
    result = np.zeros((len(x_samples), len(y_samples)))
    for i, x in enumerate(x_samples):
        for j, y in enumerate(y_samples):
            if (y>=0):
                if(x>=0): square = E2
                else: square = E1
            elif (x>=0): square = E3
            else: continue 
            for coords,fun in square.base_functions().items():   
                result[i,j] += U[vertex_index_mapping[coords]] * float(fun((x,y)))
    return result


Function to print L-Shape *u* values in user-friendly form

In [12]:
def print_LShape(matrix):
    result = ''
    for i,row in enumerate(matrix):
        row_result = ''
        for j,elem in enumerate(row):
            str_elem = '    ' if (i < len(matrix)/2 - 1 and j < len(matrix)/2 - 1) else f'{elem:.2f}'
            row_result += str_elem + '\t'
        result = row_result + '\n' + result
    print('\n' + result)

In [13]:
# Max number of points fitting on screen: 15
point_samples_num=int(input('Enter number of points: '))
g_coeff=np.float128(input('Enter coefficient for g function: '))
print_LShape(get_sample_values(n=point_samples_num, g_coeff=g_coeff))

Enter number of points: 4
Enter coefficient for g function: 3

3.35	3.35	3.35	3.35	
1.12	1.12	1.86	3.35	
0.00	0.00	1.12	3.35	
    	0.00	1.12	3.35	

